In [1]:
# Import libraries
import ee
import geemap
import geedim as gd
import pandas as pd
import geopandas as gpd
import os

c:\ProgramData\miniconda3\envs\qgismaps2025\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# Trigger the authentication flow
ee.Authenticate()

True

In [3]:
# Initialize the library
ee.Initialize()

In [4]:
# Initializes geemap/ GEE Python API
geemap.ee_initialize()

## 1. Functions for Layers Visualization

In [5]:
# Function to visualize a FeatureCollection
def FeatureMap(feature, vis_params, name):
    Map = geemap.Map(center=[9.613729, 8.736314], zoom =5)
    Map.add_basemap("SATELLITE")

    styled_layer = feature.style(**vis_params)
    
    Map.add_layer(styled_layer, {}, name)
    return Map

In [6]:
# Function to visualize a EE Image/Raster
def rasterMap(raster, viz_params, name):
    """
    """

    Map = geemap.Map(center=[9.613729, 8.736314], zoom =5)
    Map.add_basemap("SATELLITE")
    Map.add_layer(raster, viz_params, name)
    return Map

## 2. Study Extent / Regions of Interest (ROIs)

In [7]:
# Study extent from FAO GAUL
FAO_adm0 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
FAO_adm1 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
FAO_adm2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Nigerian administrative boundary
nigeria_adm0 = FAO_adm0.filter(ee.Filter.eq("ADM0_NAME", "Nigeria")) # Change to your desired country
nigeria_adm1 = FAO_adm1.filter(ee.Filter.eq("ADM0_NAME", "Nigeria")) # Change to your desired country

# Set the Nigeria adm0 as the ROI
roi = nigeria_adm0.geometry() 

# Visualize the the Nigerian administrative boundary
roi_map = FeatureMap(nigeria_adm0, {"color": "red", "fillColor": "00000000", "width": 2}, "Adm0 - Nigeria")
roi_map.addLayer(nigeria_adm1, {"color": "black", "fillColor": "00000000", "width": 2}, "Adm1- Nigeria")

roi_map

Map(center=[9.613729, 8.736314], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [8]:
# The boundary feature collection contains multiple geometry types: 'LineString', 'Polygon'
# Function to get geometry types for each Feature in the collection
def geom_to_poly(fc):
    """
    """
    # Extract the geometry type
    def geom_type(feat):
        geom = feat.geometry().type()
        return feat.set("geomType", geom)

    fc_filtered= fc.map(geom_type)

    # Filter collection to separate features with polygons as geometry and 
    # features with other types of geometry
    fc_polys = fc_filtered.filter(ee.Filter.eq("geomType", "Polygon"))
    fc_no_polys = fc_filtered.filter(ee.Filter.neq("geomType", "Polygon"))

    # See the numbers of municipalities with polygon and non-polygon geometry
    num_polys = fc_polys.size().getInfo()
    num_no_polys = fc_no_polys.size().getInfo()
    
    print(f"Number of features with polygon geometry : {num_polys}")
    print(f"Number of features with other geometries : {num_no_polys}")

    # Get the remaining polygon geometry
    # Last geometry (polygon) from the "fc_no_polys" GeometryCollection 
    # This was known after inspecting the "fc_no_polys"
    def get_geom(feat):
        geom2 = feat.geometry().geometries().get(-1)
        poly = ee.Geometry(geom2)
        return ee.Feature(poly).copyProperties(feat)


    fc_polys2 = fc_no_polys.map(get_geom)

    fc_polys2.first().geometry()
    num_polys2 = fc_polys2.size().getInfo()
    print(f"Number of features with extracted polygons from other geometries : {num_polys2}")

    # Merge  'fc_polys' and fc_polys2'
    fc_poly_all = fc_polys.merge(fc_polys2)
    
    print(f"Number of total/final feature with polygon geometry : {fc_poly_all.size().getInfo()}")
    
    return fc_poly_all


# Get the polygon geometry from the "nigeria_adm1"
nigeria_poly = geom_to_poly(nigeria_adm1)

Number of features with polygon geometry : 19
Number of features with other geometries : 18
Number of features with extracted polygons from other geometries : 18
Number of total/final feature with polygon geometry : 37


## 3. Download and Process/Aggregate Population Layer

In [10]:
# GHSL: Global Population Surfaces Image Collection
# GHSL: Spatial distribution of residential population. Population Count.
# Reference: https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP#description
ghsl_pop_collection = (ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")
            .filterDate("2020", "2025")
            .filterBounds(roi))


# Check the native projection of the GHS pop layer
ghsl_native_projection = ghsl_pop_collection.first().projection()
ghsl_native_resolution = ghsl_native_projection.nominalScale().getInfo()

print("The Native Projection of the GHSL Pop Layer: ", ghsl_native_projection.getInfo())
print("The Native Resolution/Scale of the GHSL Pop Layer: ", ghsl_native_resolution)

The Native Projection of the GHSL Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [100, 0, -18041000, 0, -100, 9000000]}
The Native Resolution/Scale of the GHSL Pop Layer:  100


In [11]:
# Direct download of GHS pop data for specific year (2025)
ghsl_pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2025").clip(roi) # Change to your desired year. The data is available at 5-years interval.

# Inspect the date of the GHS layer
ghsl_pop_date = ee.Date(ghsl_pop.get("system:time_start")).format("YYYY-MM-dd").getInfo()
print(f"The date of the GHSL pop layer is {ghsl_pop_date}")

# Visualization parameters for the GHS pop layer
vis_params_g_pop = {
  "bands" : ["population_count"],
  "min" : 0.0,
  "max": 5.0,
  "palette" : ["ffffe7", "FFc869", "ffac1d", "e17735", "f2552c", "9f0c21"]
}

# Add the GHS pop layer to the map and visualize
rasterMap(ghsl_pop.clip(roi), vis_params_g_pop, "GHSL Population - 2025")

The date of the GHSL pop layer is 2025-01-01


Map(center=[9.613729, 8.736314], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [14]:
# Aggregate the GHS population layer to desired lower resolution
def ghs_resolution_aggregate(pop_image, native_proj, new_resolution):

    """
    Aggregates a GHS population image to a lower spatial resolution using a specified scale.

    Parameters:
        pop_image (ee.Image): The GHS population image to be aggregated.
        native_proj (ee.Projection): The native projection of the input population image.
        new_resolution (float): The desired spatial resolution in meters (e.g., 500 or 1000).

    Returns:
        ee.Image: The population image aggregated to the specified lower resolution.
    
    Prints:
        The projection information and nominal scale of the aggregated image.
    """
    
    # Get the projection at the desired scale
    ghs_projection_at_new_resolution = native_proj.atScale(new_resolution)

    ghs_pop_at_new_res = pop_image.reduceResolution(
        reducer = ee.Reducer.sum().unweighted(),
        maxPixels = 1024
    ).reproject(
        crs = ghs_projection_at_new_resolution #Request the data at the scale and projection of reduced resolution (1km)
    )

    # Inspect the resolution/scale of the 'ghs_pop_at_new_res'
    ghs_pop_at_new_res_projection = ghs_pop_at_new_res.projection()
    ghs_pop_at_new_res_resolution = ghs_pop_at_new_res_projection.nominalScale().getInfo()

    print("The Native Projection of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_projection.getInfo())
    print("The Resolution/Scale of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_resolution)

    return ghs_pop_at_new_res 


# Apply function
ghsl_pop_500m = ghs_resolution_aggregate(ghsl_pop, ghsl_native_projection, 500)

The Native Projection of the Aggregated GHS Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [500, 0, -18041000, 0, -500, 9000000]}
The Resolution/Scale of the Aggregated GHS Pop Layer:  500


## 4. Export The Population Layer & Boundary Extents

In [15]:
# Function to export image to Google Drive
def img_export_to_drive(image, desc, aoi, scale, folder_name):
    img_Export = ee.batch.Export.image.toDrive(
        image = image,
        description = desc,
        folder = folder_name,
        region = aoi,
        #crs = "EPSG:32634",
        scale = scale,
        maxPixels = 1e13
        )

    img_Export.start()
    
    return img_Export

# Export folder
out_folder = "QGIS_Maps_2025"

# Export the Original GHSL population layer
#ghsl_pop_export = img_export_to_drive(ghsl_pop.clip(roi), "GHSL_Pop_Nigeria_2025_100m", roi, 100, out_folder)

# Export the Aggregated GHSL population layer
#ghsl_pop_export = img_export_to_drive(ghsl_pop_500m.clip(roi), "GHSL_Pop_Nigeria_2025_500m", roi, 500, out_folder)

In [36]:
ghsl_pop_export.status()

{'state': 'RUNNING',
 'description': 'GHSL_Pop_Nigeria_2025_500m',
 'priority': 100,
 'creation_timestamp_ms': 1744033791478,
 'update_timestamp_ms': 1744033842861,
 'start_timestamp_ms': 1744033797841,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'batch_eecu_usage_seconds': 21.195,
 'id': '3P4GJTM6KKDDSAZMC7Z3DNTZ',
 'name': 'projects/earthengine-legacy/operations/3P4GJTM6KKDDSAZMC7Z3DNTZ'}

In [10]:
# Function to export a feature collection to G Drive
def feature_export_to_drive(fc, desc, folder):
    fc_export = ee.batch.Export.table.toDrive(
                    collection = fc, 
                    description= desc, 
                    folder = folder, 
                    fileFormat = "SHP",
                    )

    fc_export.start()
    
    return fc_export

# Export folder
out_folder = "QGIS_Maps_2025"

# Export the boundary feature collection (polygon geometry)
#nigeria_adm0_export= feature_export_to_drive(nigeria_adm0, "Nigeria_Adm0", out_folder)
nigeria_adm1_export= feature_export_to_drive(nigeria_poly, "Nigeria_Adm1", out_folder)   

In [ ]:
nigeria_adm1_export.status()

{'state': 'FAILED',
 'description': 'Nigeria_Adm0',
 'priority': 100,
 'creation_timestamp_ms': 1744066667963,
 'update_timestamp_ms': 1744066674643,
 'start_timestamp_ms': 1744066672792,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.342183679,
 'error_message': "Shapefiles cannot contain multiple geometry types; found 'Point', 'LineString', 'Polygon'.",
 'id': 'Z4Y4GE2R2755YVXYZIDQ3DCM',
 'name': 'projects/134285796370/operations/Z4Y4GE2R2755YVXYZIDQ3DCM'}